<a href="https://colab.research.google.com/github/OpenMLCo/TensorRT-Yolo-Deepsort/blob/main/examples/Training_yolov4_tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Turn on GPU

# Compiling Darknet

In [1]:
# clone repo from alexeyAB darknet yolo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15363, done.
remote: Total 15363 (delta 0), reused 0 (delta 0), pack-reused 15363
Receiving objects: 100% (15363/15363), 14.00 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (10330/10330), done.


In [2]:
%cd darknet/

/content/darknet


In [3]:
# Config GPU compilation
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/AVX=0/AVX=1/g' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/g' Makefile

In [4]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -ffp-contract=fast -mavx -mavx2 -msse3 -msse4.1 -msse4.2 -msse4a -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guar

# Coping data files

In [5]:
# download from Google Drive
!gdown --id 1eh_4_TuA9u2yUA8IH9K555wZCwTOBiB6

Downloading...
From: https://drive.google.com/uc?id=1eh_4_TuA9u2yUA8IH9K555wZCwTOBiB6
To: /content/darknet/images_MOT15.zip
100% 427M/427M [00:02<00:00, 172MB/s]


In [6]:
# unzip data on data folder
!unzip 'images_MOT15.zip' -d /content/darknet/data/

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/darknet/data/images_MOT15/000525ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000835ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000478ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000012ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000168ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000923ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000999ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000527ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000698ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000254ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000689ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000534ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000129ETHB.txt  
  inflating: /content/darknet/data/images_MOT15/000591ETHB.txt  
  inflating: /content/darknet

In [7]:
#download config files
!gdown --id 1TpvqWcmJm7JFKJbp8OH4sjwBKSOhe7kA -O /content/folder.zip

Downloading...
From: https://drive.google.com/uc?id=1TpvqWcmJm7JFKJbp8OH4sjwBKSOhe7kA
To: /content/folder.zip
100% 1.29k/1.29k [00:00<00:00, 1.09MB/s]


In [8]:
!unzip /content/folder.zip -d /content/darknet/data/config/

Archive:  /content/folder.zip
  inflating: /content/darknet/data/config/obj.data  
  inflating: /content/darknet/data/config/obj.names  
  inflating: /content/darknet/data/config/yolov4-tiny-custom.cfg  


# data set-up

In [9]:
# Import some librarie
import os
import random
# Read images' name
path_an='/content/darknet/data/'
path='images_MOT15/'
names= [name for name in os.listdir(path_an+path) if name.endswith('.txt')]
print("Number of txt files:",len(names))
names= [name for name in os.listdir(path_an+path) if name.endswith('.jpg')]
print("Number of images:",len(names))

Number of txt files: 3773
Number of images: 3773


In [10]:
# Shuffle data names
save_filestxt_path = '/content/darknet/data/config/'
random.shuffle(names)
random.shuffle(names)
random.shuffle(names)
random.shuffle(names)
random.shuffle(names)
# Splitting data names into train and test
train_names = names[:int(len(names)*0.8)]
random.shuffle(train_names)
random.shuffle(train_names)
test_names = names[int(len(names)*0.8):]
random.shuffle(test_names)
random.shuffle(test_names)
# Save training names
file1 = open(save_filestxt_path+"train.txt", "w")
for image in train_names:
  file1.write(path_an+path+image+"\n")
file1.close()
# Save test names
file1 = open(save_filestxt_path+"test.txt", "w")
for image in test_names:
  file1.write(path_an+path+image+"\n")
file1.close()

# Train Yolo model

In [22]:
%cd /content/darknet

/content/darknet


In [23]:
# check GPU instance
!nvidia-smi

Mon Nov 22 03:22:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!./darknet detector train \
    /content/darknet/data/config/obj.data  \
    /content/darknet/data/config/yolov4-tiny-custom.cfg  \
    -dont_show -map